In [ ]:
#!pip install parameters

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle
import nest
nest.set_verbosity('M_ERROR')


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 19 2024 15:53:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [89]:
def get_exc_inh_matrix(val_exc, val_inh, num_pops):
    """Creates a matrix for excitatory and inhibitory values.

    Parameters
    ----------
    val_exc
        Excitatory value.
    val_inh
        Inhibitory value.
    num_pops
        Number of populations.

    Returns
    -------
    matrix
        A matrix of of size (num_pops x num_pops).

    """
    matrix = np.zeros((num_pops, num_pops))
    matrix[:, 0:num_pops:2] = val_exc
    matrix[:, 1:num_pops:2] = val_inh
    return matrix


net_dict = {
    # factor to scale the number of neurons
    "N_scaling": 0.01,
    # factor to scale the indegrees
    "K_scaling": 0.01,
    # neuron model
    "neuron_model": "iaf_psc_exp",
    # names of the simulated neuronal populations
    "populations": ["L2E", "L2I", "L3E", "L3I", "L4E", "L4I", "L5E", "L5I", "L6E", "L6I"],
    # number of neurons in the different populations (same order as
    # 'populations')
    "full_num_neurons": np.array([20683, 5834, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948]),
    # mean rates of the different populations in the non-scaled version of the
    # microcircuit (in spikes/s; same order as in 'populations');
    # necessary for the scaling of the network.
    # The values were obtained by running this PyNEST microcircuit without MPI,
    # 'local_num_threads' 4 and both 'N_scaling' and 'K_scaling' set to 1.
    #
    # Since these rates were only taken from one simulation, they alone are not sufficient for verification.
    # For that, rates should be compared to mean values over multiple runs with different RNG seeds.
    "full_mean_rates": np.array([0.903, 2.965, 0.903, 2.965, 4.414, 5.876, 7.569, 8.633, 1.105, 7.829]),
    # connection probabilities (the first index corresponds to the targets
    # and the second to the sources)
    "conn_probs": np.array(
        [
            [0.1009, 0.1689, 0.1009, 0.1689, 0.0437, 0.0818, 0.0323, 0.0, 0.0076, 0.0],
            [0.1346, 0.1371, 0.1346, 0.1371, 0.0316, 0.0515, 0.0755, 0.0, 0.0042, 0.0],
            [0.1009, 0.1689, 0.1009, 0.1689, 0.0437, 0.0818, 0.0323, 0.0, 0.0076, 0.0],
            [0.1346, 0.1371, 0.1346, 0.1371, 0.0316, 0.0515, 0.0755, 0.0, 0.0042, 0.0],
            [0.0077, 0.0059, 0.0077, 0.0059, 0.0497, 0.135, 0.0067, 0.0003, 0.0453, 0.0],
            [0.0691, 0.0029, 0.0691, 0.0029, 0.0794, 0.1597, 0.0033, 0.0, 0.1057, 0.0],
            [0.1004, 0.0622, 0.1004, 0.0622, 0.0505, 0.0057, 0.0831, 0.3726, 0.0204, 0.0],
            [0.0548, 0.0269, 0.0548, 0.0269, 0.0257, 0.0022, 0.06, 0.3158, 0.0086, 0.0],
            [0.0156, 0.0066, 0.0156, 0.0066, 0.0211, 0.0166, 0.0572, 0.0197, 0.0396, 0.2252],
            [0.0364, 0.001, 0.0364, 0.001, 0.0034, 0.0005, 0.0277, 0.008, 0.0658, 0.1443],
        ]
    ),
    # mean amplitude of excitatory postsynaptic potential (in mV)
    "PSP_exc_mean": 0.15,
    # relative standard deviation of the weight
    "weight_rel_std": 0.1,
    # relative inhibitory weight
    "g": -4,
    # mean delay of excitatory connections (in ms)
    "delay_exc_mean": 1.5,
    # mean delay of inhibitory connections (in ms)
    "delay_inh_mean": 0.75,
    # relative standard deviation of the delay of excitatory and
    # inhibitory connections
    "delay_rel_std": 0.5,
    # turn Poisson input on or off (True or False)
    # if False: DC input is applied for compensation
    "poisson_input": True,
    # indegree of external connections to the different populations (same order
    # as in 'populations')
    "K_ext": np.array([1600, 1500, 1600, 1500, 2100, 1900, 2000, 1900, 2900, 2100]),
    # rate of the Poisson generator (in spikes/s)
    "bg_rate": 8.0,
    # delay from the Poisson generator to the network (in ms)
    "delay_poisson": 1.5,
    # initial conditions for the membrane potential, options are:
    # 'original': uniform mean and standard deviation for all populations as
    #             used in earlier implementations of the model
    # 'optimized': population-specific mean and standard deviation, allowing a
    #              reduction of the initial activity burst in the network
    #              (default)
    "V0_type": "optimized",
    #thalamic poisson params
    "th_rate": 120,
    "th_start":200,
    "th_duration":10,
    
    # parameters of the neuron model
    "neuron_params": {
        # membrane potential average for the neurons (in mV)
        "V0_mean": {"original": -58.0, "optimized": [-68.28, -63.16, -68.28, -63.16, -63.33, -63.45, -63.11, -61.66, -66.72, -61.43]},
        # standard deviation of the average membrane potential (in mV)
        "V0_std": {"original": 10.0, "optimized": [5.36, 4.57, 5.36, 4.57, 4.74, 4.94, 4.94, 4.55, 5.46, 4.48]},
        # reset membrane potential of the neurons (in mV)
        "E_L": -65.0,
        # threshold potential of the neurons (in mV)
        "V_th": -50.0,
        # membrane potential after a spike (in mV)
        "V_reset": -65.0,
        # membrane capacitance (in pF)
        "C_m": 250.0,
        # membrane time constant (in ms)
        "tau_m": 10.0,
        # time constant of postsynaptic currents (in ms)
        "tau_syn": 0.5,
        # refractory period of the neurons after a spike (in ms)
        "t_ref": 2.0,
    },
}

# derive matrix of mean PSPs,
# the mean PSP of the connection from L4E to L23E is doubled
PSP_matrix_mean = get_exc_inh_matrix(
    net_dict["PSP_exc_mean"], net_dict["PSP_exc_mean"] * net_dict["g"], len(net_dict["populations"])
)
PSP_matrix_mean[0, 2] = 2.0 * net_dict["PSP_exc_mean"]

updated_dict = {
    # matrix of mean PSPs
    "PSP_matrix_mean": PSP_matrix_mean,
    # matrix of mean delays
    "delay_matrix_mean": get_exc_inh_matrix(
        net_dict["delay_exc_mean"], net_dict["delay_inh_mean"], len(net_dict["populations"])
    ),
}

net_dict.update(updated_dict)

In [51]:
def num_synapses_from_conn_probs(conn_probs, popsize1, popsize2):
    """Computes the total number of synapses between two populations from
    connection probabilities.

    Here it is irrelevant which population is source and which target.

    Parameters
    ----------
    conn_probs
        Matrix of connection probabilities.
    popsize1
        Size of first population.
    popsize2
        Size of second population.

    Returns
    -------
    num_synapses
        Matrix of synapse numbers.
    """
    prod = np.outer(popsize1, popsize2)
    num_synapses = np.log(1.0 - conn_probs) / np.log((prod - 1.0) / prod)
    return num_synapses

In [59]:
full_num_synapses = num_synapses_from_conn_probs(
            net_dict["conn_probs"], net_dict["full_num_neurons"], net_dict["full_num_neurons"]
        )

num_synapses = np.round(
            (full_num_synapses * net_dict["N_scaling"] * net_dict["K_scaling"])
        ).astype(int)

In [62]:
def postsynaptic_potential_to_current(C_m, tau_m, tau_syn):
    r"""Computes a factor to convert postsynaptic potentials to currents.

    The time course of the postsynaptic potential ``v`` is computed as
    :math: `v(t)=(i*h)(t)`
    with the exponential postsynaptic current
    :math:`i(t)=J\mathrm{e}^{-t/\tau_\mathrm{syn}}\Theta (t)`,
    the voltage impulse response
    :math:`h(t)=\frac{1}{\tau_\mathrm{m}}\mathrm{e}^{-t/\tau_\mathrm{m}}\Theta (t)`,
    and
    :math:`\Theta(t)=1` if :math:`t\geq 0` and zero otherwise.

    The ``PSP`` is considered as the maximum of ``v``, i.e., it is
    computed by setting the derivative of ``v(t)`` to zero.
    The expression for the time point at which ``v`` reaches its maximum
    can be found in Eq. 5 of [1]_.

    The amplitude of the postsynaptic current ``J`` corresponds to the
    synaptic weight ``PSC``.

    References
    ----------
    .. [1] Hanuschkin A, Kunkel S, Helias M, Morrison A and Diesmann M (2010)
           A general and efficient method for incorporating precise spike times
           in globally time-driven simulations.
           Front. Neuroinform. 4:113.
           DOI: `10.3389/fninf.2010.00113 <https://doi.org/10.3389/fninf.2010.00113>`__.

    Parameters
    ----------
    C_m
        Membrane capacitance (in pF).
    tau_m
        Membrane time constant (in ms).
    tau_syn
        Synaptic time constant (in ms).

    Returns
    -------
    PSC_over_PSP
        Conversion factor to be multiplied to a `PSP` (in mV) to obtain a `PSC`
        (in pA).

    """
    sub = 1.0 / (tau_syn - tau_m)
    pre = tau_m * tau_syn / C_m * sub
    frac = (tau_m / tau_syn) ** sub

    PSC_over_PSP = 1.0 / (pre * (frac**tau_m - frac**tau_syn))
    return PSC_over_PSP


In [66]:
def adjust_weights_and_input_to_synapse_scaling(
    full_num_neurons,
    full_num_synapses,
    K_scaling,
    mean_PSC_matrix,
    PSC_ext,
    tau_syn,
    full_mean_rates,
    DC_amp,
    poisson_input,
    bg_rate,
    K_ext,
):
    """Adjusts weights and external input to scaling of indegrees.

    The recurrent and external weights are adjusted to the scaling
    of the indegrees. Extra DC input is added to compensate for the
    scaling in order to preserve the mean and variance of the input.

    Parameters
    ----------
    full_num_neurons
        Total numbers of neurons.
    full_num_synapses
        Total numbers of synapses.
    K_scaling
        Scaling factor for indegrees.
    mean_PSC_matrix
        Weight matrix (in pA).
    PSC_ext
        External weight (in pA).
    tau_syn
        Synaptic time constant (in ms).
    full_mean_rates
        Firing rates of the full network (in spikes/s).
    DC_amp
        DC input current (in pA).
    poisson_input
        True if Poisson input is used.
    bg_rate
        Firing rate of Poisson generators (in spikes/s).
    K_ext
        External indegrees.

    Returns
    -------
    PSC_matrix_new
        Adjusted weight matrix (in pA).
    PSC_ext_new
        Adjusted external weight (in pA).
    DC_amp_new
        Adjusted DC input (in pA).

    """
    PSC_matrix_new = mean_PSC_matrix / np.sqrt(K_scaling)
    PSC_ext_new = PSC_ext / np.sqrt(K_scaling)

    # recurrent input of full network
    indegree_matrix = full_num_synapses / full_num_neurons[:, np.newaxis]
    input_rec = np.sum(mean_PSC_matrix * indegree_matrix * full_mean_rates, axis=1)

    DC_amp_new = DC_amp + 0.001 * tau_syn * (1.0 - np.sqrt(K_scaling)) * input_rec

    if poisson_input:
        input_ext = PSC_ext * K_ext * bg_rate
        DC_amp_new += 0.001 * tau_syn * (1.0 - np.sqrt(K_scaling)) * input_ext
    return PSC_matrix_new, PSC_ext_new, DC_amp_new

In [69]:
PSC_over_PSP = postsynaptic_potential_to_current(
            net_dict["neuron_params"]["C_m"],
            net_dict["neuron_params"]["tau_m"],
            net_dict["neuron_params"]["tau_syn"],
        )
PSC_matrix_mean = net_dict["PSP_matrix_mean"] * PSC_over_PSP
PSC_ext = net_dict["PSP_exc_mean"] * PSC_over_PSP
DC_amp = np.zeros(10)
PSC_matrix_mean, PSC_ext, DC_amp = adjust_weights_and_input_to_synapse_scaling(
                net_dict["full_num_neurons"],
                full_num_synapses,
                net_dict["K_scaling"],
                PSC_matrix_mean,
                PSC_ext,
                net_dict["neuron_params"]["tau_syn"],
                net_dict["full_mean_rates"],
                DC_amp,
                net_dict["poisson_input"],
                net_dict["bg_rate"],
                net_dict["K_ext"],
            )

In [70]:
net_dict['full_num_neurons']

array([20683,  5834, 20683,  5834, 21915,  5479,  4850,  1065, 14395,
        2948])

In [71]:
num_neurons = np.round((net_dict["full_num_neurons"] * net_dict["N_scaling"])).astype(int)
print(num_neurons)

[2068  583 2068  583 2192  548  485  106 1440  295]


In [72]:
use_thalamic_pulse = True #the thalamus only sends to layer 4,5,6
th_radius = 0.5
th_delay = 1.0
th_weight = 500 * 1 #what is this weight
th_rate = 100.
th_start = 500.
th_stop = 550.

In [73]:
nest.ResetKernel()
nest.rng_seed = 42
nest.resolution = 0.1
nest.local_num_threads = os.cpu_count()

In [74]:
extent = 4
positions = nest.spatial.free(
    pos=nest.random.uniform(min=-extent / 2.,
                            max=extent / 2.),
    edge_wrap=True,
    extent=[extent, extent])

In [80]:
populations = []

for i in np.arange(len(population_sizes)):
    population = nest.Create(net_dict["neuron_model"], population_sizes[i], positions=positions)

    population.set(
        tau_syn_ex=net_dict["neuron_params"]["tau_syn"],
        tau_syn_in=net_dict["neuron_params"]["tau_syn"],
        E_L=net_dict["neuron_params"]["E_L"],
        V_th=net_dict["neuron_params"]["V_th"],
        V_reset=net_dict["neuron_params"]["V_reset"],
        t_ref=net_dict["neuron_params"]["t_ref"],
        V_m=nest.random.normal(
            net_dict["neuron_params"]["V0_mean"]["optimized"][i],
            net_dict["neuron_params"]["V0_std"]["optimized"][i],
        )    
    )

    populations.append(population)

left_populations = populations
right_populations = populations


In [81]:
def connect_region_populations(region_pops):
    for i, target_pop in enumerate(region_pops):
        for j, source_pop in enumerate(region_pops):
            if num_synapses[i][j] >= 0.0:
                # Define the connection rule
                conn_dict_rec = {"rule": "fixed_total_number", "N": num_synapses[i][j]}

                # Determine weight bounds
                if PSC_matrix_mean[i][j] < 0:
                    w_min = -np.inf
                    w_max = 0.0
                else:
                    w_min = 0.0
                    w_max = np.inf

                # Synaptic parameters
                syn_dict = {
                    "synapse_model": "static_synapse",
                    "weight": nest.math.redraw(
                        nest.random.normal(
                            mean=PSC_matrix_mean[i][j],
                            std=abs(PSC_matrix_mean[i][j] * net_dict["weight_rel_std"]),
                        ),
                        min=w_min,
                        max=w_max,
                    ),
                    "delay": nest.math.redraw(
                        nest.random.normal(
                            mean=net_dict["delay_matrix_mean"][i][j],
                            std=(net_dict["delay_matrix_mean"][i][j] * net_dict["delay_rel_std"]),
                        ),
                        min=nest.resolution - 0.5 * nest.resolution,
                        max=np.inf,
                    ),
                }

                nest.Connect(source_pop, target_pop, conn_spec=conn_dict_rec, syn_spec=syn_dict)

connect_region_populations(left_populations)
connect_region_populations(right_populations)


In [84]:
#connections between two columns

layer2_populations = [left_populations[1],right_populations[1]]
layer3_populations = [left_populations[1],right_populations[1]]
layer6_populations = [left_populations[8],right_populations[8]]

connect_region_populations(layer2_populations)
connect_region_populations(layer3_populations)
connect_region_populations(layer6_populations)

In [ ]:
#poisson

ext_indegrees = np.round((net_dict["K_ext"] * net_dict["K_scaling"])).astype(int)
poisson_bg_input = nest.Create("poisson_generator", n=len(population_sizes))
poisson_bg_input.rate = net_dict["bg_rate"] * ext_indegrees

In [90]:
#thalamus

poisson_th = nest.Create("poisson_generator")
poisson_th.set(
    rate=net_dict["th_rate"],
    start=net_dict["th_start"],
    stop=(net_dict["th_start"] + net_dict["th_duration"]),
)
    
th_connections = [right_populations[4:8]]
nest.Connect(poisson_th, th_connections, syn_spec={'weight': 1.0})

TypeError: unhashable type: 'list'